## Transform ResStock Run Results to 15 SF prototypes (Elevate Energy)

In [100]:
import pandas as pd
import numpy as np
import os
os.getcwd()

'/Users/lliu2/Documents/GitHub/OpenStudio-BuildStock/files'

### Download results from S3:
://S3/resbldg-datasets/chicagoeui

In [70]:
# local path to downloaded results
result_path = '/Users/lliu2/Documents/Chicago retrofits/ResStock results/'
iteration_path = 'cookcnty_2007_06/baseline' # <----- 
baseline_result = os.path.join(result_path,iteration_path,'results_up00.parquet')

In [71]:
res = pd.read_parquet(baseline_result)
res

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_maximum_daily_use_overlap_kw,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw
0,1,11,2020-10-14 17:49:58,2020-10-14 17:51:21,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,5.824132,2.069717,1.629272,1.235172,14.090909,18.454545,11.371100,9.117308,5250.0,13.308919
1,2,26,2020-10-14 16:29:31,2020-10-14 16:31:07,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,10.723327,4.630761,4.726618,4.668860,16.545455,18.545455,12.707560,17.596156,140.0,20.320909
2,3,4,2020-10-14 17:07:42,2020-10-14 17:09:15,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,12.956261,3.223823,2.932147,2.274189,15.000000,14.818182,21.536294,15.150313,4503.0,23.897877
3,4,30,2020-10-14 17:27:57,2020-10-14 17:45:58,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,95.610663,33.536177,23.071787,17.973436,15.090909,15.181818,197.049109,127.097327,4861.0,223.678405
4,5,35,2020-10-14 16:28:37,2020-10-14 16:30:07,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,5.790440,1.663152,1.202366,1.142503,16.818182,18.000000,10.614424,7.277902,5154.0,12.125103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,19,2020-10-14 14:37:16,2020-10-14 14:38:46,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,6.027889,1.008248,1.878901,1.023454,13.636364,18.000000,14.044676,6.354918,5150.0,15.714847
39996,39997,2,2020-10-14 17:12:33,2020-10-14 17:14:05,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,6.966059,1.876124,1.469002,1.473012,15.909091,14.545455,15.729118,10.611976,4528.0,18.401095
39997,39998,2,2020-10-14 17:47:44,2020-10-14 18:21:09,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,228.101891,32.142732,26.323638,30.435582,14.000000,18.909091,346.739319,379.439890,2969.0,409.721097
39998,39999,33,2020-10-14 17:50:27,2020-10-14 17:51:49,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,9.266511,2.450066,1.441402,1.026811,10.818182,14.545455,34.131628,31.007886,4503.0,43.489856


In [99]:
proto_char = ['build_existing_model.geometry_stories',
              'build_existing_model.geometry_wall_type',
              'build_existing_model.vintage_acs'
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'ResStock - {iteration_path} - RESULT SUMMARY:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

for char in proto_char:
    Nchar = res.groupby(char)['building_id'].count()
    Nchar['None'] = len(res[res[char].isnull()])
    print(f'  * {Nchar}\n')
    

ResStock - cookcnty_2007_06/baseline - RESULT SUMMARY:

  * 39964 / 40000 samples ran successfully, 36 failed, efficacy: 99.9% 

  * build_existing_model.geometry_stories
1       21757
2       10624
3        7583
None       36
Name: building_id, dtype: int64

  * build_existing_model.geometry_wall_type
Masonry     27950
WoodStud    12014
None           36
Name: building_id, dtype: int64

  * build_existing_model.vintage_acs
1940-59     9395
1960-79    10192
1980-99     5511
2000-09     3041
2010s        467
<1940      11358
None          36
Name: building_id, dtype: int64



In [73]:
## Elevate Energy prototype chars
groups = pd.read_csv(os.path.join(result_path,'Groups.csv'))

In [74]:
groups['median gas'] = np.NAN
groups['median elec'] = np.NAN
for i, row in groups.iterrows():
    median_gas = res[res['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
    res['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
    res['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
    res['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]['simulation_output_report.total_site_natural_gas_therm'].median()
    groups.iloc[i, groups.columns.get_loc('median gas')] = median_gas

    median_elec = res[res['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
    res['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
    res['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
    res['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]['simulation_output_report.total_site_electricity_kwh'].median()
    groups.iloc[i, groups.columns.get_loc('median elec')] = median_elec


In [75]:
groups['Gas Diff(%)'] = ((groups['Non-normalized gas usage'] - groups['median gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff(%)'] = ((groups['Non-normalized elec. usage'] - groups['median elec']) / groups['Non-normalized elec. usage']) * 100

out = groups.drop(['Stories','WallType','Vintage'], axis=1)
out[['HousingGroupNo','HousingGroupName','median gas','median elec','Gas Diff(%)','Elec Diff(%)']]

,HousingGroupNo,HousingGroupName,median gas,median elec,Gas Diff(%),Elec Diff(%)
0,1,Masonry Post-1978 less than 2 stories,676.804468,8527.478970,40.264389,17.064005
1,2,Masonry All Years Split Level,993.463555,8575.688036,29.840144,11.233951
2,3,Masonry Post-1978 2 or more stories,1098.021311,10728.981652,39.402797,41.130416
3,4,Masonry 1942-1978 Less than 2 Stories,883.758940,8074.570337,26.962071,8.024031
4,5,Masonry 1942-1978 2 or more stories,1363.654066,9444.697894,4.439098,10.468311
5,6,Masonry Pre-1942 less than 2 stories,1040.526398,8631.827290,30.816064,7.174672
6,7,Masonry Pre-1942 2 or more stories,1409.110392,9600.565731,31.061135,27.663007
7,8,Frame Post-1978 Less than 2 stories,727.457008,8433.163109,30.981309,9.834672
8,9,Frame Post-1978 Split Level,898.069311,9191.513162,23.110504,12.000831
9,10,Frame Post-1978 2 or more stories,1178.008084,10680.372402,17.158363,20.856818


### Write output to folder

In [76]:
#out_name = "Groups_results" + "_" + baseline_result.split("_")[2]
out.to_csv(os.path.join(result_path, iteration_path, 'baseline_results.csv'), index=False)
print(f'file saved to:\n  {os.path.join(result_path, iteration_path)}')

file saved to:
/Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_2007_06/baseline
